In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from scipy import stats

# Ссылка на CSV файл в GitHub (raw-версия)
data_url = 'https://raw.githubusercontent.com/DerBenzz1996/Netology-Derbentsev_N.A._YCP23/main/horse_data.csv'

# Загрузка данных
data = pd.read_csv(data_url, header=None, na_values='?')  # Заменим '?' на NaN для удобства работы с пропусками

# Переименование столбцов для удобства на основе описания
data.columns = [
    "surgery", "age", "hospital_number", "rectal_temperature", "pulse", "respiratory_rate", 
    "temperature_of_extremities", "peripheral_pulse", "mucous_membranes", "capillary_refill_time", 
    "pain", "peristalsis", "abdominal_distension", "nasogastric_tube", "nasogastric_reflux", 
    "nasogastric_reflux_ph", "rectal_examination", "abdomen", "packed_cell_volume", "total_protein", 
    "abdominocentesis_appearance", "abdominocentesis_total_protein", "outcome", 
    "surgical_lesion", "type_of_lesion_1", "type_of_lesion_2", "type_of_lesion_3", "cp_data"
]

# Выбранные столбцы с созданием копии для избежания предупреждений
selected_columns = [
    "rectal_temperature", "pulse", "respiratory_rate", "temperature_of_extremities", 
    "pain", "abdominal_distension", "packed_cell_volume", "outcome"
]
selected_data = data[selected_columns].copy()  # Создаем копию выбранных столбцов

### Задание 1: Базовое изучение данных
# Расчет базовых статистических метрик
basic_stats = selected_data.describe(include='all').T  # Основные статистики для всех столбцов
modes = selected_data.mode().iloc[0]  # Моды для всех столбцов
basic_stats['mode'] = modes  # Добавим моду в таблицу

print("Базовые метрики для выбранных столбцов:\n", basic_stats)

### Задание 2: Поиск выбросов
# Определяем выбросы по Z-оценке (обычно значения > 3 считаются выбросами)
numeric_columns = ["rectal_temperature", "pulse", "respiratory_rate", "packed_cell_volume"]
outliers = {}

for col in numeric_columns:
    # Убираем NaN и сбрасываем индекс
    col_data = selected_data[col].dropna().reset_index(drop=True)
    
    # Вычисляем Z-оценку
    z_scores = np.abs(stats.zscore(col_data))
    
    # Находим выбросы по Z-оценке и сохраняем их в словарь
    outliers[col] = col_data[z_scores > 3]
    print(f"Выбросы в '{col}':\n", outliers[col])

# Гипотеза: выбросы могут быть вызваны ошибками измерений, аномалиями или экстремальными случаями.
# Решение: при наличии небольшого числа выбросов можно их удалить, иначе применить лог-трансформацию или замену на медиану.

### Задание 3: Работа с пропусками
# Количество пропусков в выбранных столбцах
missing_data = selected_data.isnull().sum()
print("\nКоличество пропусков по столбцам:\n", missing_data)

# Заполнение пропусков: числовые - медианой, категориальные - модой
for col in selected_data.columns:
    if selected_data[col].dtype == 'float64' or selected_data[col].dtype == 'int64':
        selected_data[col] = selected_data[col].fillna(selected_data[col].median())  # Числовые заполняем медианой
    else:
        selected_data[col] = selected_data[col].fillna(selected_data[col].mode()[0])  # Категориальные заполняем модой

print("\nДатафрейм без пропусков:\n", selected_data.head())
